In [116]:
# !brew install hbase
# !pip3 install thrift
# !pip3 install hbase-thrift
# !pip3 install happybase
# !start-hbase.sh
# !hbase thrift start
# table: certificate
# column family: feature

In [197]:
import happybase
import requests
import pandas as pd
import numpy as np

stock_code = '002316'
licence = '9831453184924e2ed3'
table_name = 'certificate'
column_family = 'feature'

In [195]:
def dump_history_specific_data(code, url, key_col, feature_type):
    response = requests.get(url=url)
    history = response.json()
    connection = happybase.Connection('localhost', 9090, transport='framed', protocol='compact')
    connection.open()
    table = connection.table(name=table_name)
    print(history)
    for item in history:
        rowkey = code + '_' + item[key_col]
        for col_name in item:
            if col_name != key_col:
                column_qulifier = 'feature:' + feature_type + '_' + col_name
                value = str(item[col_name])
                table.put(row=rowkey, data={column_qulifier:value})
    connection.close()

def dump_history_data(code, licence):
    dump_history_specific_data(code=code, url='http://api.mairui.club/zs/hfsjy/{}/dn/{}'.format(code, licence), key_col='d', feature_type='trade')
    dump_history_specific_data(code=code, url='http://api.mairui.club/zs/hkdj/{}/dn/{}'.format(code, licence), key_col='t', feature_type='kdj')
    dump_history_specific_data(code=code, url='http://api.mairui.club/zs/hmacd/{}/dn/{}'.format(code, licence), key_col='t', feature_type='macd')
# dump_history_data(code=stock_code, licence=licence)

[{'d': '2009-12-09', 'o': '38.00', 'h': '38.90', 'l': '35.30', 'c': '36.00', 'v': '176253', 'e': '652385864.41', 'zf': '19.15', 'hs': '73.44', 'zd': '91.49', 'zde': '17.20'}, {'d': '2009-12-10', 'o': '35.29', 'h': '39.60', 'l': '35.08', 'c': '39.60', 'v': '86646', 'e': '332194768.50', 'zf': '12.56', 'hs': '36.10', 'zd': '10.00', 'zde': '3.60'}, {'d': '2009-12-11', 'o': '41.00', 'h': '42.70', 'l': '39.80', 'c': '41.18', 'v': '156834', 'e': '646596896.90', 'zf': '7.32', 'hs': '65.35', 'zd': '3.99', 'zde': '1.58'}, {'d': '2009-12-14', 'o': '40.21', 'h': '40.63', 'l': '37.09', 'c': '37.81', 'v': '78682', 'e': '301992194.30', 'zf': '8.60', 'hs': '32.78', 'zd': '-8.18', 'zde': '-3.37'}, {'d': '2009-12-15', 'o': '38.00', 'h': '38.00', 'l': '35.60', 'c': '36.00', 'v': '63455', 'e': '229695497.16', 'zf': '6.35', 'hs': '26.44', 'zd': '-4.79', 'zde': '-1.81'}, {'d': '2009-12-16', 'o': '35.80', 'h': '35.80', 'l': '33.40', 'c': '33.80', 'v': '59827', 'e': '204479428.96', 'zf': '6.67', 'hs': '24.93'

In [202]:
connection = happybase.Connection('localhost', 9090, transport='framed', protocol='compact')
connection.open()
table = connection.table(table_name)
rows = table.scan(row_prefix=stock_code.encode('utf-8'))
df = pd.DataFrame()
idx = int(0)
for row in rows:
    date = row[0].decode('utf-8').split('_')[-1]
    cells = row[1]
    for column in cells:
        df.at[idx, column.decode('utf-8').split(':')[-1]] = np.float32(cells[column].decode('utf-8'))
    df.at[idx, 'date'] = date
    idx = idx + 1
connection.close()
df.dropna(inplace=True)
df.set_index('date')
df.sort_index(inplace=True)
df

,macd_dea,macd_diff,macd_ema12,macd_ema26,macd_macd,trade_c,trade_e,trade_h,trade_hs,trade_l,trade_o,trade_v,trade_zd,trade_zde,trade_zf,date,kdj_d,kdj_j,kdj_k
8,-0.130,-0.836,34.603001,35.439400,-1.412,31.18,90117944.0,31.690001,12.060000,30.400000,30.469999,28941.0,1.53,0.47,4.20,2009-12-21,8.500000,8.500000,8.500000
9,-0.352,-1.241,33.704102,34.944599,-1.776,28.76,114190072.0,31.080000,16.139999,28.600000,30.889999,38735.0,-7.76,-2.42,7.95,2009-12-22,7.680000,2.790000,6.050000
10,-0.561,-1.397,33.224998,34.622002,-1.672,30.59,155427968.0,31.290001,21.440001,28.690001,28.690001,51455.0,6.36,1.83,9.04,2009-12-23,8.030000,10.130000,8.730000
11,-0.737,-1.439,32.944199,34.383301,-1.405,31.40,152094352.0,31.799999,20.629999,29.389999,30.170000,49503.0,2.65,0.81,7.88,2009-12-24,9.880000,20.980000,13.580000
12,-0.891,-1.508,32.605099,34.113400,-1.234,30.74,90032200.0,31.389999,12.130000,30.420000,30.500000,29106.0,-2.10,-0.66,3.09,2009-12-25,12.130000,25.660000,16.639999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2852,0.113,0.165,5.015800,4.851300,0.104,5.54,212349872.0,5.540000,12.510000,5.010000,5.010000,394061.0,9.92,0.50,10.52,2023-07-14,65.970001,74.309998,68.750000
2853,0.129,0.196,5.099500,4.903800,0.133,5.56,599063808.0,5.940000,33.700001,5.400000,5.540000,1061622.0,0.36,0.02,9.75,2023-07-17,66.760002,71.500000,68.339996
2854,0.143,0.200,5.135000,4.935400,0.113,5.33,313424032.0,5.480000,18.760000,5.200000,5.410000,591018.0,-4.14,-0.23,5.04,2023-07-18,65.019997,54.520000,61.520000
2855,0.163,0.243,5.246500,5.003900,0.159,5.86,271282048.0,5.860000,14.860000,5.230000,5.250000,468242.0,9.94,0.53,11.82,2023-07-19,67.360001,81.459999,72.059998


In [193]:
connection = happybase.Connection('localhost', 9090, transport='framed', protocol='compact')
connection.open()
table = connection.table(table_name)
rows = table.scan(row_prefix=stock_code.encode('utf-8'))
for row in rows:
    print(row)
connection.close()
